# 数据读取

In [ ]:
import pandas as pd

In [ ]:
def read_data(file_path,all_data):
    pd_time=pd.read_csv(file_path+'/'+'Z230进口硫酸压力-Raw.csv',header="infer",encoding='gbk',index_col=False)
    pd_time.columns = ["Id", "Time", 'Value']
    pd_time['Time']=pd_time['Time'].str.replace('[','').str.replace(']','')
    pd_time['Time']=pd.to_datetime(pd_time['Time'])
    for file in files:
        data=pd.read_csv(file_path+'/'+file,header="infer",encoding='gbk')
        file_name=file.strip('-Raw.csv')
        data.columns = ["id", "Time", file_name]
        all_data=all_data.append(data[file_name])
    trans_data=all_data.T
    all_data=pd.concat([pd_time,trans_data],axis=1).drop(['Id','Value'],axis=1).set_index('Time')
    return all_data

In [ ]:
x_April_path='/Mine/new/生产参数记录表（固定时间间隔）-2018年4月'
x_May_path='/Mine/new/生产参数记录表（固定时间间隔）-2018年5月'
files=[ 'Z230进口硫酸压力-Raw.csv','Z230进口液氨压力-Raw.csv','Z230进口液氨流量-Raw.csv','Z230进口洗涤液压力-Raw.csv',
        'Z230进口洗涤液浓度-Raw.csv','成品重量-Raw.csv', '返料重量-Raw.csv','Z221进口液氨流量-Raw.csv',
        'Z220造粒机电流-Raw.csv',
        'F101鼓风机电流-Raw.csv', 'E301干燥窑转速-Raw.csv','E301干燥窑电流-Raw.csv','E301干燥窑进口烟气温度-Raw.csv',
        'F101热风炉炉膛温度-Raw.csv','F101热风炉鼓风压力-Raw.csv','L101斗提机电流-Raw.csv','S101A振网筛电流-Raw.csv',
        'S101B振网筛电流-Raw.csv','S102A破碎机电流1-Raw.csv','S102A破碎机电流2-Raw.csv','S102B破碎机电流1-Raw.csv',
        'S102B破碎机电流2-Raw.csv']

In [ ]:
all_data=pd.DataFrame()
all_April_data=read_data(x_April_path,all_data)

In [ ]:
all_data=pd.DataFrame()
all_May_data=read_data(x_May_path,all_data)
all_x_data=pd.concat([all_April_data,all_May_data],axis=0)

# 特征构造及数据清洗

In [ ]:
all_x_data['返料比']=all_x_data['返料重量']/all_x_data['成品重量']
all_x_data['总压力']=all_x_data['Z230进口洗涤液压力']+all_x_data['Z230进口液氨压力']+all_x_data['Z230进口硫酸压力']
all_x_data['含氮量']=all_x_data['Z230进口液氨压力']/all_x_data['总压力']
all_x_data['含磷量']=all_x_data['Z230进口洗涤液压力']/all_x_data['总压力']
all_x_data['硫酸量']=all_x_data['Z230进口硫酸压力']/all_x_data['总压力']
all_x_data['中和度']=all_x_data['Z230进口洗涤液压力']/all_x_data['Z230进口液氨压力']

In [ ]:
April_y_data=pd.read_csv('/home/data/2.产品检验报告/产品检验报告2018-4-1.csv')
May_y_data=pd.read_csv('/home/data/2.产品检验报告/产品检验报告2018-5-1-sample.csv')
y_data=pd.concat([April_y_data,May_y_data],axis=0,sort=True)
y_data=pd.concat([y_data, pd.DataFrame(columns=['0_time','1_time','2_time'])],sort=True)
y_data.reset_index(drop=True,inplace=True)

In [ ]:
for i in range(0,len(y_data)):
    sp_str=y_data['product_batch'][i].split(' ')
    ssp_str_0=sp_str[1].split(':')
    ssp_str_1=sp_str[3].split(':')
    if int(ssp_str_0[0])>int(ssp_str_1[0]):
        ssp_str_1[0]=int(ssp_str_1[0])+23
    else:
        ssp_str_1[0]=int(ssp_str_1[0])-1
    y_data['0_time'][i]=str(2018.)+sp_str[0] + ' ' + ssp_str_0[0] + ':00'+':00'
    y_data['2_time'][i]=str(2018.)+sp_str[0]+' '+str(ssp_str_1[0])+':59'+':59' 

In [ ]:
y_data['0_time']=pd.to_datetime(y_data['0_time'],format='%Y.%m.%d %H:%M:%S')
y_data['2_time']=pd.to_datetime(y_data['2_time'],format='%Y.%m.%d %H:%M:%S')

In [ ]:
re_x=pd.DataFrame()
std_x=pd.DataFrame()
for j in range(1,len(y_data)):#
    start=y_data['0_time'][j]-pd.Timedelta("7 Hours")
    end=y_data['2_time'][j]-pd.Timedelta("7 Hours")
    resample=all_x_data[start:end]
    drop_index=resample[(resample['成品重量']<=5)|(resample['返料重量']<=50)].index
    clean_data=resample.drop(index=drop_index)
    sample=pd.DataFrame(clean_data[start:end].mean()).T
    sample['Time']=start+pd.Timedelta("7 Hours")
    sample.index=[j-1]
    re_x=re_x.append(sample)

In [ ]:
add_data=y_data[['nitrogen_content','phosphorus_content','total_nutrient','water_content','particle_size']][0:143]
per_x_data=pd.concat([re_x,add_data],axis=1) 

In [ ]:
all_use_columns=['Z230进口液氨流量','Z220造粒机电流','F101鼓风机电流','E301干燥窑电流','F101热风炉鼓风压力',
                 'S101A振网筛电流','S101B振网筛电流','E301干燥窑转速','S102A破碎机电流1','S102A破碎机电流2',
                 'S102B破碎机电流1','S102B破碎机电流2','Z230进口液氨压力', 'Z221进口液氨流量','返料比', '总压力',
                 '含氮量', '含磷量', '中和度','硫酸量','E301干燥窑进口烟气温度','F101热风炉炉膛温度',
                 'nitrogen_content', 'phosphorus_content', 'total_nutrient','water_content','particle_size']
scaled_x=per_x_data[all_use_columns]/per_x_data[all_use_columns].mean()

In [ ]:
x1_cols=[ 'Z221进口液氨流量','Z220造粒机电流', 'E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','nitrogen_content','E301干燥窑进口烟气温度']
x2_cols=['Z221进口液氨流量','Z220造粒机电流','E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','phosphorus_content','E301干燥窑进口烟气温度']
x3_cols=[ 'Z221进口液氨流量','Z220造粒机电流','E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','total_nutrient','E301干燥窑进口烟气温度']
x4_cols=[ 'Z221进口液氨流量','Z220造粒机电流','E301干燥窑电流', 'F101热风炉鼓风压力','返料比',
         '总压力', '含氮量', '含磷量', '中和度', '硫酸量','water_content','E301干燥窑进口烟气温度']
x5_cols=['S102A破碎机电流1','返料比','总压力', '含氮量','含磷量','中和度','硫酸量','E301干燥窑进口烟气温度']

In [ ]:
x1=scaled_x[x1_cols][0:143].values
x2=scaled_x[x2_cols][0:143].values
x3=scaled_x[x3_cols][0:143].values
x4=scaled_x[x4_cols][0:143].values
x5=scaled_x[x5_cols][37:143].values

In [ ]:
per_y_data=y_data[['nitrogen_content','phosphorus_content','total_nutrient','water_content','particle_size']][1:144]
per_y_data.columns = ["y1", "y2", 'y3','y4','y5']
per_y_data.reset_index(drop=True,inplace=True)

In [ ]:
y1=per_y_data['y1'][0:143].values
y2=per_y_data['y2'][0:143].values
y3=per_y_data['y3'][0:143].values
y4=per_y_data['y4'][0:143].values
y5=per_y_data['y5'][37:143].values

# 模型训练

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import numpy as np
from sklearn.model_selection import KFold
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])      
        self.meta_model.fit(oof_train,y)
        return self
   
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR

In [ ]:
knr_1=KernelRidge()
ela_1=ElasticNet(alpha=5e-05,l1_ratio=0.8,max_iter=10000)
gbr_1= GradientBoostingRegressor(learning_rate=0.1, max_depth=1, n_estimators=80)
xg_1=XGBRegressor(learning_rate=0.2, max_depth=1, n_estimators=40)
svr_1=SVR(C=100, gamma=0.1, kernel='rbf')
stack_model_1 = stacking(mod=[ela_1,gbr_1,xg_1,svr_1],meta_model=knr_1)
stack_model_1.fit(x1,y1)
for m in range(142,220):
    if m<143:   
        test_x1=scaled_x[x1_cols][m:m+1].values
        test_y=stack_model_1.predict(test_x1).tolist()
        scaled_x.loc[m+1,'nitrogen_content']=round(test_y[0],2)
    else:    
        test_x1=scaled_x[x1_cols][m:m+1].values
        test_x1[0][10]=test_x1[0][10]/16.3
        test_y=stack_model_1.predict(test_x1).tolist()
        scaled_x.loc[m+1,'nitrogen_content']=test_y[0]

In [ ]:
knr_2=KernelRidge()
ela_2=ElasticNet(alpha= 5e-05, l1_ratio= 0.8, max_iter= 10000)
gbr_2= GradientBoostingRegressor(learning_rate= 0.05, max_depth= 4, n_estimators= 40)
xg_2=XGBRegressor(learning_rate= 0.1, max_depth= 1, n_estimators= 80)
svr_2=SVR(C= 10, gamma= 0.5, kernel= 'rbf')
stack_model_2 = stacking(mod=[ela_2,gbr_2,xg_2,svr_2],meta_model=knr_2)
stack_model_2.fit(x2,y2)
for m in range(142,220):
    if m<143:   
        test_x2=scaled_x[x2_cols][m:m+1].values
        test_y=stack_model_2.predict(test_x2).tolist()
        scaled_x.loc[m+1,'phosphorus_content']=round(test_y[0],2)
    else:    
        test_x2=scaled_x[x2_cols][m:m+1].values
        test_x2[0][10]=test_x2[0][10]/44.47
        test_y=stack_model_2.predict(test_x2).tolist()
        scaled_x.loc[m+1,'phosphorus_content']=test_y[0]

In [ ]:
knr_3=KernelRidge()
ela_3=ElasticNet(alpha= 5e-05, l1_ratio= 0.8, max_iter= 10000)
gbr_3= GradientBoostingRegressor(learning_rate= 0.02, max_depth= 3, n_estimators= 100)
xg_3=XGBRegressor(learning_rate= 0.1, max_depth= 1, n_estimators= 80)
svr_3=SVR(C= 10, gamma= 0.1, kernel= 'rbf')
stack_model_3 = stacking(mod=[ela_3,gbr_3,xg_3,svr_3],meta_model=knr_3)
stack_model_3.fit(x3,y3)
for m in range(142,220):
    if m<143:   
        test_x3=scaled_x[x3_cols][m:m+1].values
        test_y=stack_model_3.predict(test_x3).tolist()
        scaled_x.loc[m+1,'total_nutrient']=round(test_y[0],2)
    else:    
        test_x3=scaled_x[x3_cols][m:m+1].values
        test_x3[0][10]=test_x3[0][10]/60.792
        test_y=stack_model_3.predict(test_x3).tolist()
        scaled_x.loc[m+1,'total_nutrient']=test_y[0]

In [ ]:
knr_4=KernelRidge()
ela_4=ElasticNet(alpha= 5e-05, l1_ratio= 0.8, max_iter= 10000)
gbr_4= GradientBoostingRegressor(learning_rate= 0.1, max_depth= 1, n_estimators= 60)
xg_4=XGBRegressor(learning_rate= 0.05, max_depth= 1, n_estimators= 200)
svr_4=SVR(C= 1, gamma= 0.5, kernel= 'rbf')
stack_model_4 = stacking(mod=[ela_4,gbr_4,xg_4,svr_4],meta_model=knr_4)
stack_model_4.fit(x4,y4)
for m in range(142,220):
    if m<143:   
        test_x4=scaled_x[x4_cols][m:m+1].values
        test_y=stack_model_4.predict(test_x4).tolist()
        scaled_x.loc[m+1,'water_content']=round(test_y[0],2)
    else:    
        test_x4=scaled_x[x4_cols][m:m+1].values
        test_x4[0][10]=test_x4[0][10]/2.798
        test_y=stack_model_4.predict(test_x4).tolist()
        scaled_x.loc[m+1,'water_content']=test_y[0]

In [ ]:
knr_5=KernelRidge()
ela_5=ElasticNet(alpha= 0.0005, l1_ratio= 0.8, max_iter= 10000)
gbr_5= GradientBoostingRegressor(learning_rate= 0.1, max_depth= 2, n_estimators= 20)
xg_5=XGBRegressor(learning_rate= 0.05, max_depth= 1, n_estimators= 200)
svr_5=SVR(C= 100, gamma= 1, kernel= 'rbf')
stack_model_5 = stacking(mod=[ela_5,gbr_5,xg_5,svr_5],meta_model=knr_5)
stack_model_5.fit(x5,y5)
for m in range(142,220): 
    test_x5=scaled_x[x5_cols][m:m+1].values
    test_y=stack_model_5.predict(test_x5).tolist()
    scaled_x.loc[m+1,'particle_size']=test_y[0]

# 结果输出

In [ ]:
final_columns=['nitrogen_content','phosphorus_content','total_nutrient','water_content','particle_size']
result=scaled_x[final_columns][143:220]

In [ ]:
output=pd.DataFrame({"id":id,"phosphorus_content":result['phosphorus_content'].values,
                     "nitrogen_content":result['nitrogen_content'].values,
                     'total_nutrient':result['total_nutrient'].values,
                     'water_content':result['water_content'].values,
                     'particle_size':result['particle_size'].values})

In [ ]:
output.set_index('id',inplace=True)

In [ ]:
output.to_csv('/home/code/result.csv',encoding='UTF-8')